In [2]:
import cv2
import os
import pandas as pd
import Augmentor
import numpy as np

# Train

In [3]:
data=[]
labels = []
classes = 43
cur_path = os.getcwd()
cur_path

'/home/teresababio/FinalProject/Traffic_sign_classification/datos'

In [4]:
os.path.exists(cur_path)

True

In [5]:
np.array(([[1,2],[2,3]])).tolist()

[[1, 2], [2, 3]]

In [6]:
for i in range(classes):
    path = os.path.join(cur_path,'Train',str(i))
    images = os.listdir(path)
    for img in images:
        img2=img
        try:
            img_path = os.path.join(path, img)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (50,50), interpolation = cv2.INTER_CUBIC)/255
            data.append(img)
            labels.append(i)
        except:
            print(f"Error al leer {img2} {i}")

Error al leer output 0
Error al leer output 3
Error al leer output 4
Error al leer output 5
Error al leer output 6
Error al leer output 7
Error al leer output 8
Error al leer output 9
Error al leer output 11
Error al leer output 14
Error al leer output 15
Error al leer output 16
Error al leer output 17
Error al leer output 18
Error al leer output 19
Error al leer output 20
Error al leer output 21
Error al leer output 22
Error al leer output 23
Error al leer output 24
Error al leer output 25
Error al leer output 26
Error al leer output 27
Error al leer output 28
Error al leer output 29
Error al leer output 30
Error al leer output 31
Error al leer output 32
Error al leer output 33
Error al leer output 34
Error al leer output 35
Error al leer output 36
Error al leer output 37
Error al leer output 39
Error al leer output 40
Error al leer output 41
Error al leer output 42


In [7]:
type(np.array([1,2]))

numpy.ndarray

In [8]:
np.amax(data[0])

1.0

In [9]:
train = pd.DataFrame(columns= ["label", "image"])
train["label"] = labels
train["image"] = data

In [10]:
train.head()

,label,image
0,0,"[[[0.44313725490196076, 0.4, 0.360784313725490..."
1,0,"[[[0.6470588235294118, 0.3176470588235294, 0.3..."
2,0,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,..."
3,0,"[[[0.6745098039215687, 0.1450980392156863, 0.1..."
4,0,"[[[0.7019607843137254, 0.6352941176470588, 0.6..."


In [11]:
train.shape

(39209, 2)

In [12]:
train["label"].value_counts()

2     2250
1     2220
13    2160
12    2100
38    2070
10    2010
4     1980
5     1860
25    1500
9     1470
7     1440
3     1410
8     1410
11    1320
35    1200
18    1200
17    1110
31     780
14     780
33     689
15     630
26     600
28     540
23     510
30     450
16     420
34     420
6      420
36     390
22     390
40     360
20     360
21     330
39     300
29     270
24     270
41     240
42     240
32     240
27     240
37     210
19     210
0      210
Name: label, dtype: int64

## Data Augmentation

In [ ]:
for label,value in zip(train["label"].value_counts().index, train["label"].value_counts().values) :
    if 2000>value:
        print(label, value)
        path = os.path.join(cur_path,'Train',str(label), 'output')
        if not os.path.exists(path):
            print("Creacion de carpeta output")
            p = Augmentor.Pipeline(os.path.join(cur_path,'Train',str(label)))
            p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
            p.zoom(probability=0.3, min_factor=1.1, max_factor=1.6)
            p.skew(probability=0.6)
            p.sample(2000-value)  
        
 
        images = os.listdir(path)
        for img in images:
            img2=img
            try:
                img_path = os.path.join(path, img)
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (50,50), interpolation = cv2.INTER_CUBIC)/255
                data.append(img)
                labels.append(label)
            except:
                print(f"Error al leer {img2}")
        
        


In [ ]:
train = pd.DataFrame(columns= ["label", "image"])
train["label"] = labels
train["image"] = data 
train["label"].value_counts()

# Test

In [ ]:
test = pd.read_csv( 'Test.csv')

labels = test["ClassId"].values



data =[]
path = os.path.join(cur_path,'Test')
images = os.listdir(path)

for img in images:
        img2= img
        try:
            img_path = os.path.join(path, img)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (50,50), interpolation = cv2.INTER_CUBIC)/255
            data.append(img)
        except:
            print(f"Error al leer {img2}")








In [ ]:
test = pd.DataFrame(columns= ["label", "image"])
test["label"] = labels
test["image"] = data

In [ ]:
test.shape

In [ ]:
test["label"].value_counts()

In [ ]:
#test.to_csv("data_test.csv", index = False)
#train.to_csv("data_train.csv", index = False)

# Modelo 

In [13]:
from tensorflow.keras.applications import VGG19

2022-12-22 13:21:46.108686: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-22 13:21:46.412145: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/teresababio/miniconda3/envs/core/lib/python3.9/site-packages/cv2/../../lib64:
2022-12-22 13:21:46.412174: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-22 13:21:49.672116: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnv

In [14]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam

In [15]:
#Avoid OOM errors by setting PGU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

2022-12-22 13:21:52.459063: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/teresababio/miniconda3/envs/core/lib/python3.9/site-packages/cv2/../../lib64:
2022-12-22 13:21:52.459188: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-22 13:21:52.459246: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-FNGN17J8): /proc/driver/nvidia/version does not exist


In [16]:
vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(50,50,3))
vgg19.summary()

2022-12-22 13:21:52.916633: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50, 50, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 50, 50, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 50, 50, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 25, 25, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 25, 25, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 25, 25, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

In [17]:
vgg_model = tf.keras.Sequential([VGG19(weights='imagenet', include_top=False, input_shape=(50,50,3)),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Flatten(),
                                 keras.layers.Dense(512, activation='sigmoid'),
                                 keras.layers.Dense(43, activation='softmax')
                                ])
vgg_model.layers[0].trainable=True
vgg_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1, 1, 512)         20024384  
                                                                 
 batch_normalization (BatchN  (None, 1, 1, 512)        2048      
 ormalization)                                                   
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dense_1 (Dense)             (None, 43)                22059     
                                                                 
Total params: 20,311,147
Trainable params: 20,310,123
Non-trainable params: 1,024
________________________________________

In [18]:
vgg_model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [19]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
Y_enc = ohe.fit_transform(np.array(train["label"]).reshape(-1, 1)).todense()

In [20]:
X = list(train["image"])

In [21]:
type(X)

list

In [22]:
Y_enc.shape

(39209, 43)

In [ ]:
vgg_model.fit(X, Y_enc,
                        epochs=10,
                        )

In [23]:
data= np.array(data)
labels = np.array(labels)

In [24]:
from keras.utils import to_categorical

In [25]:
#Converting the labels into one hot encoding
y_train = to_categorical(labels, 43)

In [ ]:
vgg_model.fit(data, y_train,
                        epochs=10,
                        )